In [2]:

!pip install altair vega_datasets --quiet

import pandas as pd
import altair as alt


alt.data_transformers.disable_max_rows()


archivo = 'BASE DE DATOS DE CONCIERTOS.editada.xlsx'
df = pd.read_excel(archivo, sheet_name='Hoja1')


df = df[['AÑO', 'ENTRADA MÁS BARATA', 'ENTRADA MÁS CARA']].dropna()
df.columns = df.columns.str.strip().str.lower()

df['entrada más barata'] = pd.to_numeric(df['entrada más barata'], errors='coerce')
df['entrada más cara'] = pd.to_numeric(df['entrada más cara'], errors='coerce')


df_grouped = df.groupby('año').agg(
    min_entrada=('entrada más barata', 'min'),
    max_entrada=('entrada más cara', 'max')
).reset_index()


df_grouped['año'] = pd.to_datetime(df_grouped['año'], format='%Y')


df_long = df_grouped.melt(id_vars='año',
                          value_vars=['min_entrada', 'max_entrada'],
                          var_name='Tipo de entrada',
                          value_name='Precio')


df_long['Tipo de entrada'] = df_long['Tipo de entrada'].map({
    'min_entrada': 'Mín. de ENTRADA MÁS BARATA',
    'max_entrada': 'Máx. de ENTRADA MÁS CARA'
})


line_chart = alt.Chart(df_long).mark_line(point=True).encode(
    x=alt.X('año:T', title='Año'),
    y=alt.Y('Precio:Q', title='Precio (USD)'),
    color=alt.Color('Tipo de entrada:N', scale=alt.Scale(range=['green', 'purple'])),
    tooltip=['año:T', 'Tipo de entrada:N', 'Precio:Q']
).properties(
    title='Evolución de precios de entradas (mín y máx)',
    width=700,
    height=400
)


text = alt.Chart(df_long).mark_text(align='left', dx=5, dy=-5).encode(
    x='año:T',
    y='Precio:Q',
    text=alt.Text('Precio:Q', format="$.2f"),
    color=alt.Color('Tipo de entrada:N', title='Tipo de entrada')
)


(line_chart + text).configure_title(fontSize=18)



alt.LayerChart(...)